# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [16]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)                         # That simulates the web browser    
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [17]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,"Reporting & Data Visualization, Product Analyt...",National Basketball Association (NBA),"Secaucus, NJ"
1,Willing to move back to India?,Samsung Electronics,"San Jose, CA"
2,Remote Epidemiologist,Insight Global,Denver Metropolitan Area
3,Data Analyst,Mondo,"San Francisco, CA"
4,Programming Analyst,Sony Pictures Entertainment,"Miami, FL"
5,Insights Analyst,Daymon,"Hutchinson, KS"
6,Data Scientist,Apple,"Cupertino, CA"
7,Data and Analytics Engineer,rprt,"Buffalo, NY"
8,Data Analyst - Risk and Payments,Wish,"San Francisco, CA"
9,Data Analyst,Motion Recruitment,"Boston, MA"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [79]:
# copy and paste the code in order to add changes to add: 
## +1 Pages search

def scrape_linkedin_job_search_pages(keywords, num_pages):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    ## [ADDED] Create a for loop for each page (if we only want the 1st one --> 0)
    
    ## [MODYFIED POSITION] Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    for page in range(num_pages): 
        if page < 1:
            # Assemble the full url with parameters
            scrape_url = ''.join([BASE_URL, 'keywords=', keywords])
        else:
            scrape_url = ''.join([BASE_URL, 'keywords=', keywords, f"&start={25*page}"])

        # Create a request to get the data from the server 
        page = requests.get(scrape_url)                         # That simulates the web browser    
        soup = BeautifulSoup(page.text, 'html.parser')

        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
        # Inject job titles, companies, and locations into the "empty dataframe"
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data = data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data

In [86]:
# Call the function

results = scrape_linkedin_job_search_pages('data%20analysis', 5)
results

## Add some validation when there are not as many pages available as we provide as an input argument

,Title,Company,Location
0,Willing to move back to India?,Samsung Electronics,"San Jose, CA"
1,Remote Epidemiologist,Insight Global,Denver Metropolitan Area
2,Data Analyst,Mondo,"San Francisco, CA"
3,Programming Analyst,Sony Pictures Entertainment,"Miami, FL"
4,Data Scientist,Apple,"Cupertino, CA"
...,...,...,...
120,Research Analyst,Ragnar Research Partners,"Oklahoma City, OK"
121,Data and Insights Analyst,Blueair,"Chicago, IL"
122,Digital Analyst (20 hours / week),Amy's Kitchen,"Petaluma, CA"
123,"Data Analyst, Gaming Studio",Andiamo Partners,"New York, NY"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [97]:
# copy and paste the code in order to add changes to add: 
## +1 Pages search
### Filter by country

def scrape_linkedin_job_search_pages_country(keywords, num_pages, country):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    ## [ADDED] Create a for loop for each page (if we only want the 1st one --> 0)
    ## [MODYFIED POSITION] Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    for page in range(num_pages): 
        ## If statement to check which page we are browsing
        if page < 1:
            # Assemble the full url with parameters
            scrape_url = ''.join([BASE_URL, 'keywords=', 
                                  keywords, 
                                  f"&location={country.capitalize()}"])
        else:
            scrape_url = ''.join([BASE_URL, 'keywords=', 
                                  keywords, 
                                  f"&location={country.capitalize()}", 
                                  f"&start={25*page}"])
        print(scrape_url)
        # Create a request to get the data from the server 
        page = requests.get(scrape_url)                         # That simulates the web browser    
        soup = BeautifulSoup(page.text, 'html.parser')

        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
        # Inject job titles, companies, and locations into the "empty dataframe"
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data = data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data

In [98]:
# Call the function

results = scrape_linkedin_job_search_pages_country('data%20analysis', 3, "GERmany")
results[1:50]

https://www.linkedin.com/jobs/search/?keywords=data%20analysis&location=Germany
https://www.linkedin.com/jobs/search/?keywords=data%20analysis&location=Germany&start=25
https://www.linkedin.com/jobs/search/?keywords=data%20analysis&location=Germany&start=50


,Title,Company,Location


## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [91]:
# copy and paste the code in order to add changes to add: 
## +1 Pages search
### Filter by country
#### Adding days filter. We need a funtion that converts days to seconds. 

def scrape_linkedin_job_search_pages_country_days(keywords, num_pages, country, num_days):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    ## [ADDED] Create a for loop for each page (if we only want the 1st one --> 0)
    ## [MODYFIED POSITION] Create an empty dataframe with the columns consisting of the information you want to capture
    #### [ADDED] New column for days
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    for page in range(num_pages): 
        ## If statement to check which page we are browsing
        if page < 1:
            # Assemble the full url with parameters
            scrape_url = ''.join([BASE_URL, 
                                  f"f_TPR=r{num_days * 86400}",
                                  "&keywords=", keywords, 
                                  f"&location={country.capitalize()}"])
        else:
            scrape_url = ''.join([BASE_URL, 
                                  f"f_TPR=r{num_days * 86400}",
                                  "&keywords=", keywords, 
                                  f"&location={country.capitalize()}", 
                                  f"&start={25*page}"])
        print(scrape_url)
        # Create a request to get the data from the server 
        page = requests.get(scrape_url)                         # That simulates the web browser    
        soup = BeautifulSoup(page.text, 'html.parser')

        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
    
        # Inject job titles, companies, and locations into the "empty dataframe"
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data = data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data

In [92]:
# Call the function

results = scrape_linkedin_job_search_pages_country_days('data%20analysis', 3, "aUsTrAlia", 1)
results

https://www.linkedin.com/jobs/search/?f_TPR=r86400&keywords=data%20analysis&location=Australia
https://www.linkedin.com/jobs/search/?f_TPR=r86400&keywords=data%20analysis&location=Australia&start=25
https://www.linkedin.com/jobs/search/?f_TPR=r86400&keywords=data%20analysis&location=Australia&start=50


,Title,Company,Location
0,City Launcher - Brisbane,Bolt,"Sydney, New South Wales, Australia"
1,"Consultant, Data Analytics",Fractal,"Sydney, New South Wales, Australia"
2,Data Analyst,Hudson,"Canberra, Australian Capital Territory, Australia"
3,Data Analyst,Hydrogen Group,"Sydney, New South Wales, Australia"
4,Reporting Analyst,Nine,"Sydney, New South Wales, Australia"
...,...,...,...
70,Data Analyst,Optus,"Sydney, New South Wales, Australia"
71,(Junior) Data Scientist / Data Analyst (w/m/d),KPMG Deutschland,"Dingee, Victoria, Australia"
72,Data Analyst,Canberra Consulting,"Canberra, Australian Capital Territory, Australia"
73,Research and Insights Analyst,CBS,"Sydney, New South Wales, Australia"


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [ ]:
# your code here